In [2]:
import folium
import pandas as pd
import json
import requests

In [160]:
projects= r'P3_GrantExport.csv'
state_data = pd.read_csv(projects, sep=';')
state_data.set_index(state_data.columns[0], inplace=True)
print(state_data.shape)
state_data = state_data[['University', 'Approved Amount']]
state_data.dropna(subset=['University'], inplace=True)
print(state_data.shape)
state_data = state_data[state_data['University'] != 'Nicht zuteilbar - NA'] #Nicht zuteilbar = NA University
state_data = state_data[state_data['University'] != 'NPO (Biblioth., Museen, Verwalt.) - NPO'] #NPO = is not university
state_data

(63969, 14)
(50988, 2)


,University,Approved Amount
"﻿""Project Number""",,
4,Université de Genève - GE,41022.00
6,Universität Basel - BS,52627.00
8,Université de Fribourg - FR,53009.00
9,Université de Fribourg - FR,25403.00
10,Universität Zürich - ZH,47100.00
11,Université de Lausanne - LA,25814.00
13,Université de Genève - GE,360000.00
14,Université de Fribourg - FR,153886.00
18,Université de Genève - GE,116991.00


In [170]:
def searchCantonGeoNames(university): #returns in which canton the university is in
    query  = university.split(" - ")
    url = 'http://api.geonames.org/searchJSON?country=CH&maxRows=1&username=jnls'
    payload = {'q': query[0] }
    req = requests.get(url, params=payload)
    req_json = json.loads(req.text)
    if req_json['totalResultsCount'] > 0:
        return req_json['geonames'][0]['adminCode1']
    elif req_json['totalResultsCount'] == 0 and len(query) > 1:
        payload = {'q': query[1] }
        req = requests.get(url, params=payload)
        req_json = json.loads(req.text)
        if req_json['totalResultsCount'] > 0:
            return req_json['geonames'][0]['adminCode1']
        else:
            return "NaN"
def search_cantons(data):
    UniCanton = data['University'].drop_duplicates().to_frame()
    UniCanton['Canton'] = ''
    for i in range(len(UniCanton)):
        canton = searchCantonGeoNames(UniCanton['University'].iloc[i])
        UniCanton['Canton'].iloc[i] = canton
    #data = data[data['Canton'] != 'NaN'] 
    return UniCanton

temp = search_cantons(state_data)
temp

,University,Canton
"﻿""Project Number""",,
4,Université de Genève - GE,GE
6,Universität Basel - BS,BS
8,Université de Fribourg - FR,FR
10,Universität Zürich - ZH,ZH
11,Université de Lausanne - LA,VD
30,Universität Bern - BE,BE
39,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",ZH
48,Université de Neuchâtel - NE,NE
60,ETH Zürich - ETHZ,ZH


In [171]:
temp.set_index(temp.University, inplace=True, drop=True)
temp = temp.to_dict()['Canton']
state_data['Canton'] = state_data['University'].map(temp)
state_data.reset_index(drop=True, inplace=True)
grants = state_data.drop('University', 1)
grants['Approved Amount'] = grants['Approved Amount'].convert_objects(convert_numeric=True)
grants_sum = grants.groupby(grants.Canton).sum()
grant_canton = grants_sum['Approved Amount'].to_dict()
grant_canton

/Users/dmartr/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


{'AG': 115428285.87,
 'BE': 1519373034.6900001,
 'BL': 42771914.120000005,
 'BS': 1352251464.3699999,
 'FR': 457526170.25000012,
 'GE': 1838237308.4999998,
 'LU': 41925889.309999995,
 'NE': 383204582.29999995,
 'NaN': 639625010.44999993,
 'SG': 84229059.450000003,
 'TI': 24040082.149999999,
 'VD': 2358606670.5999999,
 'ZH': 3643993421.1700001}

In [172]:
cantonMap = pd.DataFrame({'Canton':['JU', 'ZH', 'BE', 'LU', 'SG', 'NE', 'VD', 'VS', 'GE', 'TI', 'SO', 'UR', 'OW', 'NW', 'SZ', 'GL', 'ZG', 'BL', 'BS', 'FR', 'SH', 'AR', 'AI', 'GR', 'AG', 'TG']})
cantonMap['Grants'] = 0
cantonMap['Grants'] = cantonMap['Canton'].map(grant_per_canton)

ch_geo = r'ch-cantons.topojson.json'

map = folium.Map(location=[46.82244,8.22410], zoom_start=8)
map.choropleth(data=cantonMap, 
                columns=['Canton', 'Grants'], 
                geo_path=ch_geo,
                topojson='objects.cantons', 
                fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                key_on='feature.id')
map.save('ch-cantons.html')

/Users/dmartr/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.
